In [17]:
import numpy as np
import scipy.linalg as la

# Finding the Initial Point (provided)

In [18]:
def starting_point(A, b, c):
    """Calculate an initial guess to the solution of the linear program
    min c^T x, Ax = b, x>=0.
    Reference: Nocedal and Wright, p. 410.
    """
    # Calculate x, lam, mu of minimal norm satisfying both
    # the primal and dual constraints.
    B = la.inv(A.dot(A.T))
    x = A.T.dot(B.dot(b))
    lam = B.dot(A.dot(c))
    mu = c - A.T.dot(lam)
    
    # Perturb x and s so they are nonnegative.
    dx = max((-3./2)*x.min(), 0)
    dmu = max((-3./2)*mu.min(), 0)
    x += dx*np.ones_like(x)
    mu += dmu*np.ones_like(mu)
    
    # Perturb x and mu so they are not too small and not too dissimilar.
    dx = .5*(x*mu).sum()/mu.sum()
    dmu = .5*(x*mu).sum()/x.sum()
    x += dx*np.ones_like(x)
    mu += dmu*np.ones_like(mu)
    
    return x, lam, mu

# Constructing Interior Point Solver

In [19]:
def interiorPoint(A,b,c,niter=20,tol=1e-16):
    m,n = A.shape
    sigma = 1/10
    row1 = np.hstack((np.zeros((n,n)),A.T,np.eye(n)))
    row2 = np.hstack((A,np.zeros((m,m+n))))
    DF1 = np.vstack((row1,row2))
    x, lam, mu = starting_point(A, b, c)
    duality = (x@mu)/n
    iteration = 0
    
    def vector_F(x,lam,mu):
        row1 = A.T@lam+mu-c
        row2 = A@x-b
        row3 = np.diag(mu)@x
        F = np.hstack((row1,row2,row3))
        return F
    
    while duality >= tol and iteration < niter:
        # Find direction
        row3 = np.hstack((np.diag(mu),np.zeros((n,m)),np.diag(x)))
        DF = np.vstack((DF1,row3))
        bvec = np.hstack((np.zeros(n+m),duality*sigma*np.ones(n)))-vector_F(x,lam,mu)
        direction = la.lu_solve(la.lu_factor(DF),bvec)
       
        # Find step size
        del_x = direction[:n]
        del_lam = direction[n:-n]
        del_mu = direction[-n:]
        alpha_max = min(1,np.min(-mu[del_mu<0]/del_mu[del_mu<0]))
        delta_max = min(1,np.min(-x[del_x<0]/del_x[del_x<0]))
        alpha = min(1,0.95*alpha_max)
        delta = min(1,0.95*delta_max)

        # Next point
        x = x+delta*del_x
        lam = lam+alpha*del_lam
        mu = mu+alpha*del_mu
        duality = (x@mu)/n
        iteration += 1
        print('Duality:', duality, 'on iteration', iteration)
        
    return x, c@x

# Testing

In [20]:
def randomLP(m,n):
    """Generate a linear program min c^T x s.t. Ax = b, x>=0.
    First generate m feasible constraints, then add
    slack variables to convert it into the above form.
    Inputs:
    m (int >= n): number of desired constraints.
    n (int): dimension of space in which to optimize.
    Outputs:
    A ((m,n+m) ndarray): Constraint matrix.
    b ((m,) ndarray): Constraint vector.
    c ((n+m,), ndarray): Objective function with m trailing 0s.
    x ((n,) ndarray): The first 'n' terms of the solution to the LP.
    """
    A = np.random.random((m,n))*20 - 10
    A[A[:,-1]<0] *= -1
    x = np.random.random(n)*10
    b = np.zeros(m)
    b[:n] = A[:n,:].dot(x)
    b[n:] = A[n:,:].dot(x) + np.random.random(m-n)*10
    c = np.zeros(n+m)
    c[:n] = A[:n,:].sum(axis=0)/n
    A = np.hstack((A, np.eye(m)))
    return A, b, -c, x

m, n = 7, 5
A, b, c, x = randomLP(m, n)
point, value = interiorPoint(A, b, c)
print(np.allclose(x, point[:n]))

Duality: 0.463320949873 on iteration 1
Duality: 0.101424537586 on iteration 2
Duality: 0.0204064470219 on iteration 3
Duality: 0.00364355921276 on iteration 4
Duality: 0.00110451797748 on iteration 5
Duality: 0.000203806949413 on iteration 6
Duality: 2.95529821505e-05 on iteration 7
Duality: 4.28522300846e-06 on iteration 8
Duality: 6.2135771426e-07 on iteration 9
Duality: 9.0096871522e-08 on iteration 10
Duality: 1.30640463927e-08 on iteration 11
Duality: 1.8942867271e-09 on iteration 12
Duality: 2.7467157543e-10 on iteration 13
Duality: 3.98273784374e-11 on iteration 14
Duality: 5.77496987342e-12 on iteration 15
Duality: 8.37370631646e-13 on iteration 16
Duality: 1.21418741589e-13 on iteration 17
Duality: 1.76057175304e-14 on iteration 18
Duality: 2.5528290419e-15 on iteration 19
Duality: 3.70160211076e-16 on iteration 20
True


# Problem 5